In [ ]:
#Mount drive to save files there
#clone the repository to access files from there
#pull the latest
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
! git clone https://github.com/NASAARSET/VIIRS_NOAA.git
! git -C VIIRS_NOAA/ pull

In [ ]:
! pip install netCDF4
import numpy as np
import pandas as pd
import sys
from netCDF4 import Dataset
import matplotlib.pyplot as plt
#Colab requires specific installation of cartopy
!apt-get -qq install python-cartopy python3-cartopy;
!pip uninstall -y shapely;    # cartopy and shapely aren't friends (early 2020)
!pip install shapely --no-binary shapely;
import cartopy.crs as ccrs
#from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.axes import Axes
from cartopy.mpl.geoaxes import GeoAxes
GeoAxes._pcolormesh_patched = Axes.pcolormesh
from textwrap import wrap
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
#!/usr/bin/python
# =============================================================================
# Inputs
#This uses the file "fileList.txt", containing the list of files, in order to read the file

try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    fileList = open('VIIRS_NOAA/fileList.txt', 'r')
except:
    print('Did not find a text file containing file names (perhaps name does not match)')
    sys.exit()

#loops through all files listed in the text file
for FILE_NAME in fileList:
    FILE_NAME=FILE_NAME.strip()
    #change 'raw_input' to 'input' if an error is shown about the input
    user_input=input('\nWould you like to process\n' + FILE_NAME + '\n\n(Y/N)')
    if(user_input == 'N' or user_input == 'n'):
        print('Skipping...')
        continue
    else:
        file = Dataset('VIIRS_NOAA/'+ FILE_NAME, 'r')
# read the data
        ds=file
        #grp='PRODUCT'        
        lat= ds.variables['Latitude'][:][:]
        lon= ds.variables['Longitude'][:][:]
        if 'AOD' in FILE_NAME:
            #The user has a choice of 3 sds variable and has to input a number to choose.
            #The loop keeps repeating until the user inputs a value between 1-3 inclusive.
            while  True:
              choice = input("""Pick the number with the corresponding sds variable of your choice: 
              1) AOD550
              2) QCAll
              3) AerMdl """)
              
              if choice in ['1', '2', '3']:
                break
              else:
                print("Please input a valid response!")


            if choice == '1':
              sds_name='AOD550'
            elif choice =='2':
              sds_name='QCAll'
            elif choice =='3':
              sds_name='AerMdl'

        data= ds.variables[sds_name]      
        map_label = sds_name
        map_label = map_label.replace('_', ' ')
        map_label = '\n'.join(wrap(map_label, 40))
        #get necessary attributes 
        fv=data._FillValue
        
        #get lat and lon information 
        min_lat=np.min(lat)
        max_lat=np.max(lat)
        min_lon=np.min(lon)
        max_lon=np.max(lon)

        #get valid range for AOD SDS
        range=data.getncattr("valid_range")
        print(range)
        min_range=min(range)
        max_range=max(range)
        print(max_range)
        
        #get data within valid range
        dataArray=np.array(data[:][:])
        dataArray = np.multiply(dataArray, 1.0)
        fv = fv*1.0
        dataArray[dataArray==fv]=np.nan
        data=dataArray
        valid_data = data.ravel()
        valid_data=[x for x in valid_data if x>=min_range]
        valid_data=[x for x in valid_data if x<=max_range]
        valid_data=np.asarray(valid_data)
        #find the average
        average=sum(valid_data)/len(valid_data)
        #find the standard deviation
        stdev=np.std(valid_data)
        #print information
        print('\nThe valid range of values is: ',round(min_range,3), ' to ',round(max_range,3),'\nThe average is: ',round(average,3),'\nThe standard deviation is: ',round(stdev,3))
        print('The range of latitude in this file is: ',min_lat,' to ',max_lat, 'degrees \nThe range of longitude in this file is: ',min_lon, ' to ',max_lon,' degrees')
        

        #asks user if they want to set PM2.5 calculation parameters
        user_input=input('\nWould you like to enter a slope and intercept for PM 2.5 calculation?')
        if user_input == 'Y' or user_input == 'y': 
            slope=input('Please enter a slope: ')
            intercept=input('Please enter an intercept: ')
        else:
            #if not, choose the following:
            slope=29.4
            intercept=8.8
        valid_data=data
        pm25=float(slope)*valid_data+float(intercept)
        
        
        
        #Asks user if they would like to see a map
        is_map=input('\nWould you like to create a map of this data? Please enter Y or N \n')
        #if user would like a map, view it
        if is_map == 'Y' or is_map == 'y':
            
            #turn fillvalues to NaN
            data=pm25.astype(float)
            data[np.logical_and(data>=0,data <= 12)]=0
            data[np.logical_and(data>12,data <= 35.4)]=1
            data[np.logical_and(data>35.4,data <= 55.4)]=2
            data[np.logical_and(data>55.4,data <= 150.4)]=3
            data[np.logical_and(data>150.4,data <= 250.4)]=4
            data[data>250.4]=5
            data[data < 0] = np.nan
            
            #create the map
            data = np.ma.masked_array(data, np.isnan(data))
            
            extent = (min_lon, max_lon, min_lat, max_lat)
            m = plt.axes(projection=ccrs.PlateCarree())
            m.set_extent(extent)
            
            #divider = make_axes_locatable(m)
            #cax = divider.append_axes("right", size="5%", pad=0.05)

            my_cmap=LinearSegmentedColormap.from_list('mycmap', ['green','yellow','orange','red','purple','brown'],6)
            im = plt.pcolormesh(lon, lat, data,cmap=my_cmap, transform=ccrs.PlateCarree())
            plt.clim(0,6)
             
            #create colorbar
            cb = plt.colorbar(shrink = 0.5) #The shrink determines how long the colorbar is. Increase/decrease to your liking.
            cb.set_label('AQI Category')
            cb.set_ticks([.5, 1.5,2.5,3.5,4.5,5.5])  # force there to be only 7 ticks
            cb.set_ticklabels(['Good', 'Moderate', 'Unhealthy for \nSensitive Groups','Unhealthy','Very Unhealthy','Hazardous'])  # put text labels on them
            
            m.coastlines()

            
            """
            grd = m.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=2, color='gray', alpha=0.5, linestyle='--')
            grd.xlabels_top = None
            grd.ylabels_right = None
            grd.xformatter = LONGITUDE_FORMATTER
            grd.yformatter = LATITUDE_FORMATTER
            """
            plt.autoscale()
            
            #title the plot
            plotTitle=FILE_NAME[:-4]
            plt.title('{0}\n {1}'.format(plotTitle, 'PM 2.5'), verticalalignment = 'top')
            fig = plt.gcf()
            # Show the plot window.
            plt.show()
            
            #once you close the map it asks if you'd like to save it
            is_save=str(input('\nWould you like to save this map? Please enter Y or N \n'))
            if is_save == 'Y' or is_save == 'y':
                #saves as a png if the user would like5
                pngfile = '{0}.png'.format(plotTitle, 'PM 2.5')
                fig.savefig('/content/drive/My Drive/Colab Notebooks/' + pngfile)
            
            
